In [1]:
%load_ext autoreload
%autoreload 2

In [44]:
import pandas as pd
import numpy as np
from sklearn import set_config
set_config(display='diagram')

TypeError: set_config() got an unexpected keyword argument 'display'

In [3]:
df = pd.read_csv('../raw_data/clean_data.csv', index_col='Unnamed: 0')

In [6]:
X = df.drop(columns=['enterpriseValue'])
y = df['enterpriseValue']

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [23]:
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import OneHotEncoder

In [25]:
ohe = OneHotEncoder(sparse=False)

sector_ohe = ohe.fit_transform(X_train[['sector']])
country_ohe = ohe.fit_transform(X_train[['country']])

In [33]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression

In [48]:
X_train

,sector,country,returnOnCapitalEmployed,revenue,grossProfit,ebitda,growth_rate,ebitda_margin
1762,Industrials,US,0.194301,6704.037000,3603.571000,668.443000,0.061777,0.099708
118,Consumer Cyclical,HK,0.097948,421.293429,96.506858,62.255185,0.113213,0.147772
3287,Packaging,IN,-0.044891,174.451536,65.449079,21.128247,0.015230,0.121112
4879,Energy,US,0.011467,847.118000,294.263000,114.220000,-0.049269,0.134834
1831,Industrials,US,0.033855,3683.200000,851.100000,385.800000,0.047313,0.104746
...,...,...,...,...,...,...,...,...
3570,Healthcare,US,0.133701,785.368000,605.393000,229.964000,0.095767,0.292811
3560,Technology,US,0.047917,11503.000000,4035.000000,2692.000000,0.060013,0.234026
5091,Industrials,US,0.141866,2194.226000,1192.103000,588.493000,0.067664,0.268201
4891,Basic Materials,CA,-0.129516,82.954448,-0.283197,9.421692,-0.137181,0.113577


In [54]:
roic_pipe = Pipeline([('impute', FunctionTransformer(lambda df: 0 if df['returnOnCapitalEmployed']<=0 else x)),
                      ('scaler', RobustScaler())])

num_pipe = Pipeline([('impute',SimpleImputer()),
                     ('scaler', RobustScaler())])

preprocessing = ColumnTransformer([('roic', roic_pipe, ['returnOnCapitalEmployed']),
                                   ('num', num_pipe, ['ebitda', 'revenue', 'grossProfit', 'ebitda_margin', 'growth_rate']),
                                   ('cat', OneHotEncoder(), ['sector', 'country'])])

pipe = Pipeline([('preprocessing', preprocessing),
                 ('model', LinearRegression())])

pipe

Pipeline(memory=None,
         steps=[('preprocessing',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('roic',
                                                  Pipeline(memory=None,
                                                           steps=[('impute',
                                                                   FunctionTransformer(accept_sparse=False,
                                                                                       check_inverse=True,
                                                                                       func=<function <lambda> at 0x7f8371fcaee0>,
                                                                                       inv_kw_args=None,
                                                                                       inverse_func=No

In [39]:
X_train = X_train.drop(columns=['symbol', 'description'])

In [51]:
X_train['returnOnCapitalEmployed']

1762    0.194301
118     0.097948
3287   -0.044891
4879    0.011467
1831    0.033855
          ...   
3570    0.133701
3560    0.047917
5091    0.141866
4891   -0.129516
5323   -0.155831
Name: returnOnCapitalEmployed, Length: 3776, dtype: float64

In [55]:
pipe.fit(X_train, y_train)

TypeError: float() argument must be a string or a number, not 'DataFrame'

In [29]:
ohe.

country_ohe

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [4]:
from Company_Valuation.utils import vectorize

In [5]:
vectorize(df)

,symbol,enterpriseValue,sector_x,country_x,returnOnCapitalEmployed,revenue,grossProfit,ebitda,growth_rate,ebitda_margin,...,transmission,transport,transportation,treatment,unit,utility,vehicle,water,wholesale,worldwide
0,0001.HK,21597.256799,Industrials,HK,0.051154,38402.729002,21451.105507,12705.530323,0.157184,0.330850,...,0.000000,0.0,0.075099,0.000000,0.0,0.000000,0.0,0.308243,0.0,0.065311
1,0002.HK,5136.485887,Utilities,HK,0.047146,10908.819796,3330.785385,2378.104992,0.023875,0.217998,...,0.248657,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000
2,0003.HK,4062.479373,Utilities,HK,0.091021,5217.793781,2417.570978,1608.588899,0.082469,0.308289,...,0.000000,0.0,0.000000,0.148707,0.0,0.075375,0.0,0.132763,0.0,0.000000
4,0006.HK,10.274256,Utilities,HK,0.080384,173.121215,173.121215,933.929875,0.007559,5.394659,...,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000
5,0007.HK,7.441258,Technology,HK,0.226738,69.095143,3.704255,126.968486,0.329895,1.837589,...,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7324,ZUMZ,756.986000,Consumer Cyclical,US,0.141312,978.617000,335.936000,91.338000,0.047911,0.093334,...,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000
7325,ZWM.SW,0.257718,Construction,CH,0.028899,59.782185,38.785970,3.714651,-0.051630,0.062136,...,0.000000,0.0,0.000000,0.178319,0.0,0.000000,0.0,0.000000,0.0,0.000000
7327,ZYDUSWELL.NS,1595.492210,Food Products,IN,0.034736,118.015351,75.959475,27.258775,0.189511,0.230977,...,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000
7329,ZYXI,243.379830,Healthcare,US,0.517374,45.472000,36.658000,11.844000,0.405848,0.260468,...,0.000000,0.0,0.000000,0.116434,0.0,0.000000,0.0,0.000000,0.0,0.000000


In [58]:
from Company_Valuation.trainer import Trainer

In [83]:
trainer = Trainer()

In [60]:
scaler = RobustScaler()
reg = LinearRegression()

In [72]:
from sklearn.neighbors import KNeighborsRegressor
knn_reg = KNeighborsRegressor()

In [99]:
pipe = trainer.train(knn_reg)

Cross Validated MAE = 7190.732188469275


In [97]:
trainer.evaluate(pipe)

{'mae': 6792.146337040669,
 'mse': 341479490.0709237,
 'rmse': 18479.163673470823}